In [1]:
%matplotlib inline
import numpy as np
import glob
from os.path import join, basename
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker

## Updated Tasks to Output to DB and/or FITS

In [ ]:
main_dir = '/project/bootcamp/cslage/e2v_fits_files/satellite/20200902-injectdata'
#main_dir = '/project/bootcamp/cslage/e2v_fits_files/satellite/20200831-injectdata'

## Input for script
infiles = sorted(glob.glob(join(main_dir, '*dark_dark*.fits')))
sensor_id = 'Davis_test'
manufacturer = 'e2v'
namps = 16
database = 'test.db'

seg_names = ['C17', 'C16', 'C15', 'C14', 'C13', 'C12', 'C11', 'C10',
             'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07']

In [16]:
with db_session(database) as f:
    
    ## Check if sensor exists
    try:
        sensor = Sensor.from_db(f, name=sensor_id)
        namps = sensor.namps
    except NoResultFound:
        sensor = Sensor(name=sensor_id, manufacturer=manufacturer, namps=namps)
        sensor.segments = [Segment(name=seg_names[i], amplifier_number=i+1) for i in range(namps)]
        sensor.add_to_db(f)
        f.commit()
        
    for infile in infiles:
        ccd = MaskedCCD(infile)

        for agg in sensor.segments:

            aggarr = ccd.unbiased_and_trimmed_image(agg.amplifier_number).getImage().getArray()

            signal = np.mean(aggarr[:, 307])
            if signal < 1000:
                continue

            ## Find aggressor amp and signal level
            ly = 200
            lx = 50
            y, x = (2000, col)
            aggressor_stamp = make_stamp(aggarr, y, x, ly=ly, lx=lx)

            ## Calculate crosstalk for each victim amp
            for vic in sensor.segments:
                vicarr = ccd.unbiased_and_trimmed_image(vic.amplifier_number).getImage().getArray()
                victim_stamp = make_stamp(vicarr, y, x, ly=ly, lx=lx)
                res = crosstalk_fit(aggressor_stamp, victim_stamp, noise=7.0,
                                        num_iter=1, nsig=7.0)

                result = Result(aggressor_id=agg.id,
                                aggressor_signal=signal,
                                coefficient=res[0],
                                error=res[4],
                                method='MODEL_LSQ',
                                victim_id=vic.id,
                                filename=basename(infile))
                result.add_to_db(f)